In [64]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import psycopg2

import json
import logging
import re

# from sql_queries import *

In [5]:
# URL for all laptops on B&H website
theurl = 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/'

In [91]:
# laptop table insert query
laptop_table_insert = """INSERT INTO laptop 
                        VALUES (%(name)s,
                                %(brand)s,
                                %(sku)s,
                                %(price)s,
                                %(url)s,
                                %(availability)s,
                                %(review_num)s)
                            ON CONFLICT (sku)
                            DO UPDATE SET name = excluded.name,
                                          brand = exluded.brand,
                                          price = excluded.price,
                                          url = excluded.url,
                                          availability = excluded.availability,
                                          review_num = excluded.review_num;
                      """

In [23]:
def get_page_info(soup):
    """
    This function page information from each page.
    
    input:
    soup - beautiful soup object
    
    return:
    page - page info
    """
    # get all script snippets of type 'text/javascript'
    js_info = soup.find_all("script", attrs={"type":"text/javascript"})
    
    which_page = None
    
    # loop over each of them, looking for page info
    for s in js_info:
        try:
            which_page = re.search('\"page\" : (\d+),', s.text).group(1)
            break
        except:
            pass
    return int(which_page) if which_page else -1

In [29]:
def is_last_page(soup, curr_page):
    """
    Determine whether the scraped page is the last page.
    If it's the last page, B&H will redirect scraper to page 1.
    
    input:
    soup - beautiful soup object
    curr_page - the current scraping page
    
    return:
    is_last_page - True/False
    """
    if curr_page != 1 and get_page_info(soup) <= 1:
        return True
    else:
        return False

In [76]:
def iter_laptop_from_page(soup):
    """
    Retrieve laptop related information from the page.
    - Product Name
    - Product Brand
    - Product SKU
    - Product Price
    - Product URL
    - Product Availability
    - Product # of Reviews
    
    input:
    soup - beautiful soup object
    
    output:
    page_info_dict - dictionary of each product
    """
    # return a list of laptop info from "itemDetail" divs
    products = soup.find_all('div', attrs={'data-selenium':'itemDetail'})
    
    for product in products:
        try:
            # empty dictionary
            product_dict = {}

            # get product name
            ProductName = product.find("span", attrs={'itemprop':'name'}).text.strip()
            product_dict['name'] = ProductName

            # get product brand
            ProductBrand = product.find("span", attrs={'itemprop':'brand'}).text.strip()
            product_dict['brand'] = ProductBrand

            # get product sku
            ProductInfoDict = json.loads(product['data-itemdata'])
            product_dict['sku'] = ProductInfoDict['sku']

            # get product price
            try: 
                product_dict['price'] = float(ProductInfoDict['price'])
            except:
                product_dict['price'] = None

            # get product URL
            product_dict['url'] = product.find("a", attrs={"class":"itemImg"})['href']


            # get product availability info
            ProductAvalability = product.find("div", attrs={"data-selenium":"salesComments"}).text.strip()
            product_dict['availability'] = ProductAvalability

            # get number of reviews
            try:
                reviews_str = product.find("a", attrs={'data-selenium':'itemReviews'}).text.strip()
                reviews_str = re.findall(re.compile(r'\((\d+)\)'), 'reviews')[0]
                reviews_int = int(reviews_str)
            except:
                reviews_int = 0
            product_dict['review_num'] = reviews_int
        
        except:
            logging.info('An Error Ocurred for product {}.'.format(ProductName))
            continue
            
        yield product_dict
        

In [77]:
def iter_laptop_from_site():
    """
    It's a generator function.
    It scrapes data from URL 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292',
    and return product information.
    
    input:
    page_size - the number of pages in total to search on B&H laptop section
    
    output:
    product_info - a dictionary contains all necessary info for one product  
    """
    # B&H laptop URL
    url = 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/'
    # scrape from the first page
    page = 1
    
    while True:
        # make request for each page, get source code and parse with BeautifulSoup
        try:
            req = Request(url+str(page), headers = {'User-Agent':'Mozilla/5.0'})
            thepage = urlopen(req).read()
            page_soup = BeautifulSoup(thepage, 'html.parser')
        except Exception as e:
            logging.info(f"ERROR ocurred when scraping data for page {page}.")
            logging.info(f"ERROR message: {e}")
            break
            
        # if last page is reached, exit
        if is_last_page(page_soup, page):
            logging.info("Finished scraping all the data.")
            break
        
        #parse data on this page
        product_info = iter_laptop_from_page(page_soup)
        
        # break loop if product_info is None
        if not product_info:
            break
            
        yield from product_info
        
        # move to the next page
        page += 1
            

In [78]:
def process_data(cur, conn):
    """
    - Scrape laptop data
    
    - Store it into bnhlaptop database
    
    input: 
    cur - cursor variable
    conn - database connection
    """
    # get all laptop info and save it as a list
    all_laptop_iter = iter_laptop_from_site()
    
    # insert into laptop table in batch
    psycopg2.extras.execute_batch(cursor, laptop_table_insert, all_laptop_iter)
    conn.commit()
    

In [79]:
def main():
    """
    - Connect to sparkifydb database
    
    - get cursor variable
    
    - process data stored in file path 'data/song_data' & 'data/log_data'
    
    - close database connection
    
    input: None
    return: None
    """
    # connect to database
    conn = psycopg2.connect(host='localhost', 
                            dbname='bnhlaptop', 
                            password='test', 
                            port=5432, 
                            user='postgres')
    cur = conn.cursor()
    
    # process data
    process_data(cur, conn)

    # close database connection
    conn.close()

In [80]:
all_laptop_iter = iter_laptop_from_site()

In [101]:
next(all_laptop_iter)

{'name': '13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray)',
 'brand': 'Apple',
 'sku': '1560527',
 'price': 1399.0,
 'url': 'https://www.bhphotovideo.com/c/product/1560527-REG/apple_mxk52ll_a_13_3_macbook_pro_with.html',
 'availability': 'In Stock',
 'review_num': 0}

In [92]:
conn = psycopg2.connect(host='localhost', 
                        dbname='bnhlaptop', 
                        password='test', 
                        port=5432, 
                        user='postgres')
cur = conn.cursor()

In [93]:
cur.execute(laptop_table_insert, next(all_laptop_iter))

InvalidTextRepresentation: invalid input syntax for type integer: "16" MacBook Pro (Late 2019, Space Gray)"
LINE 2:                         VALUES ('16" MacBook Pro (Late 2019,...
                                        ^


In [102]:
conn = psycopg2.connect(host='localhost', 
                        dbname='bnhlaptop', 
                        password='test', 
                        port=5432, 
                        user='postgres')
cur = conn.cursor()
cur.execute("""INSERT INTO laptop 
                        VALUES (
                                %(review_num)s,
                                %(sku)s);
                      """, next(all_laptop_iter))

NotNullViolation: null value in column "sku" violates not-null constraint
DETAIL:  Failing row contains (0, 1553853, null, null, null, null, null, null).


In [ ]:
cur.execute("""INSERT INTO laptop 
                        VALUES (%(name)s,
                                %(brand)s,
                                %(sku)s,
                                %(price)s,
                                %(url)s,
                                %(availability)s,
                                %(review_num)s)
                            ON CONFLICT (sku)
                            DO UPDATE SET name = excluded.name,
                                          brand = exluded.brand,
                                          price = excluded.price,
                                          url = excluded.url,
                                          availability = excluded.availability,
                                          review_num = excluded.review_num;
                      """, next(all_laptop_iter))